# Importacion de API key

In [7]:
import os
import json

# 1) Calculamos la ruta absoluta de la carpeta “proyecto”
#    Partimos de que os.getcwd() = .../proyecto/notebooks
ruta_notebook = os.getcwd()
ruta_proyecto = os.path.abspath(os.path.join(ruta_notebook, ".."))

In [8]:
import sys
import os

# 1) Calculamos la ruta absoluta de ../src respecto al notebook
ruta_src = os.path.abspath(os.path.join(os.getcwd(), "..", "src"))

# 2) Añadimos esa ruta al sys.path (si no está ya)
if ruta_src not in sys.path:
    sys.path.append(ruta_src)

# 3) Ahora sí podemos importar
from config import API_KEY_OWM, API_KEY_SG

In [9]:
# Comprobamos que funcionan (puedes imprimirlas)
print("OWM key:", API_KEY_OWM)
print("SG  key:", API_KEY_SG)

OWM key: 0b87f738ea444ed359d9338b6fab8ecb
SG  key: b73c5ca4-3e4e-11f0-976d-0242ac130006-b73c5d4e-3e4e-11f0-976d-0242ac130006


# Definir la zona geográfica y la resolución de la malla. 

## Mapa para seleccionar zona

In [5]:
# — Celda 1: Mostrar el mapa con Draw (solo rectángulo) —
import folium
from folium.plugins import Draw

lat, lon, zoom = 40.35, 0.40, 10
m = folium.Map(location=[lat, lon], zoom_start=zoom, tiles='OpenStreetMap')

folium.TileLayer(
    tiles='https://tiles.openseamap.org/seamark/{z}/{x}/{y}.png',
    attr='© OpenSeaMap contributors',
    name='Seamarks',
    overlay=True,
    control=True
).add_to(m)

folium.raster_layers.WmsTileLayer(
    url='https://ows.emodnet-bathymetry.eu/wms?',
    layers='emodnet:contours',
    fmt='image/png',
    transparent=True,
    version='1.3.0',
    attr='© EMODnet Bathymetry',
    name='Contornos (EMODnet)',
    overlay=True,
    control=True,
    opacity=1.0
).add_to(m)

draw = Draw(
    export=True,
    filename='selection.geojson',
    position='topright',
    draw_options={
        'polyline': False,
        'polygon': False,
        'circle': False,
        'marker': False,
        'circlemarker': False,
        'rectangle': True,
    },
    edit_options={'edit': False}
)
draw.add_to(m)

folium.LayerControl().add_to(m)
m


## Leer zona seleccionada y establecer limites

In [10]:
# 2) Construimos la ruta absoluta hasta data/raw/selection.geojson
path_geojson = os.path.join(ruta_proyecto, "data", "raw", "selection.geojson")


In [11]:
# 3) Comprobamos (opcional) que el fichero existe
if not os.path.isfile(path_geojson):
    raise FileNotFoundError(f"No se encontró el fichero en:\n{path_geojson}")

In [12]:
# 4) Abrimos y leemos el GeoJSON
with open(path_geojson, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 5) Extraemos los límites del rectángulo
coords = data['features'][0]['geometry']['coordinates'][0]
lon1, lat1 = coords[0]
lon2, lat2 = coords[2]

lat_min = min(lat1, lat2)
lat_max = max(lat1, lat2)
lon_min = min(lon1, lon2)
lon_max = max(lon1, lon2)

print("Área de estudio:")
print(f"  lat_min = {lat_min:.6f}, lat_max = {lat_max:.6f}")
print(f"  lon_min = {lon_min:.6f}, lon_max = {lon_max:.6f}")


Área de estudio:
  lat_min = 38.753227, lat_max = 40.663704
  lon_min = 1.980978, lon_max = 4.819924
